In [ ]:
!pip install folium==0.2.1
!pip install pytorch-pretrained-biggan

In [ ]:
import torch
import numpy as np
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal)

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
#import logging
## Seems to be needed, not sure why (one_hot_from_names?)
import nltk
nltk.download('wordnet')

#logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
model = BigGAN.from_pretrained('biggan-deep-256')

# Prepare a input
truncation = 0.4
class_vector = one_hot_from_names(['soap bubble', 'coffee', 'cock', 'boathouse'], batch_size=4)
noise_vector = truncated_noise_sample(truncation=truncation, batch_size=4, seed=11)

# All in tensors
noise_vector = torch.from_numpy(noise_vector)
class_vector = torch.from_numpy(class_vector)

# If you have a GPU, put everything on cuda
noise_vector = noise_vector.to('cuda')
class_vector = class_vector.to('cuda')
model.to('cuda')

# Generate an image
with torch.no_grad():
    output0 = model(noise_vector, class_vector, truncation)

# If you have a GPU put back on CPU
output0 = output0.to('cpu')

# If you have a sixtel compatible terminal you can display the images in the terminal
# (see https://github.com/saitoha/libsixel for details)
#display_in_terminal(output)

# Save results as png images
save_as_images(output0);

In [ ]:
def rescale_parameters(model, weights_only = True, mult_noise = True, alpha = 0.3, add_noise = False, sigma = 0.0, noise_dist = 'normal', normalize = False):
    c = 0

    if noise_dist=="laplace":
        m = torch.distributions.laplace.Laplace(0, 1)
    # We can also set m to the normal distribution, avoiding many lines of code below.

    with torch.no_grad():
        for name, W in model.named_parameters():
            if weights_only and not 'weight' in name:
                continue
            c = c + 1
            if mult_noise:
                if noise_dist == 'normal':
                    W.copy_(W*(1 + alpha*torch.randn(W.shape, device='cuda')))
                elif noise_dist == 'laplace':
                    W.copy_(W*(1 + alpha*m.sample(W.shape).to('cuda') ))  
                else:
                    raise Exception("Noise distribution not implemented: "+noise_dist)
                if normalize:
                    W.copy_(W/np.sqrt(1+alpha**2))
            if add_noise:
                if noise_dist == 'normal':
                    W.copy_(W + sigma*torch.randn(W.shape, device='cuda') )
                elif noise_dist == 'laplace':
                    W.copy_(W + sigma*m.sample(W.shape).to('cuda'))
                else:
                    raise Exception("Noise distribution not implemented: "+noise_dist)
                if normalize:
                    varW  = torch.var(W)
                    coeff = torch.sqrt(varW/(varW + sigma**2))
                    W.copy_(W*coeff)

    print(f"Number of layers affected: {c}.")


netGAN = BigGAN.from_pretrained('biggan-deep-256')
netGAN.to('cuda')

torch.manual_seed(8)

rescale_parameters(netGAN, alpha = 0.25, normalize = False, add_noise = False, sigma=0.0)#, noise_dist='laplace')#, weights_only=False)

with torch.no_grad():
    output = netGAN(noise_vector, class_vector, truncation)

output = output.to('cpu')
save_as_images(output);

pic_size = torch.numel(output)
l2_dist_norm = torch.sqrt( torch.sum( (output - output0)**2)/pic_size )
print(l2_dist_norm)